<a href="https://colab.research.google.com/github/NicholasSetiadi99/Churn_analysis_script/blob/main/Churn_Analysis_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import sqlalchemy
import pymysql
import pandas as pd
pymysql.install_as_MySQLdb()
import MySQLdb
import numpy as np

In [ ]:
pd.set_option('display.max_rows', 1000)

db_connection = MySQLdb.connect(host='11.111.111.111', database='mock_database', user='nicholas_test', password="xxxxx")

db_cursor = db_connection.cursor()

months = (1,2,3,4,5,6,7,8,9,10,11,12)

column_names = ["Month", "UserID", "MonthlyDeposit", "CumulativeDeposit", "MonthlyWithdrawal", "CumulativeWithdrawal", "fullWithdrawal?"]

query = '''
select %s, u.email, ifnull(monthlyDepAmt, 0), ifnull(cumDepAmt, 0),  ifnull(monthlyWitAmt, 0), ifnull(cumWitAmt, 0), ifnull(fullWit, 0)
from(select email, id, MONTH(date_created) as "Month", YEAR(date_created) as "Year"
from USER
where MONTH(date_created) <= %s
and YEAR(date_created) = 2021
and kyc_status = "VERIFIED"
and type = "C"
and status = "ACTIVE"
and id in (select user_id from DEPOSIT where status in ("APPROVED", "BOOKED"))) u
left join
(select user_id, sum(amount) as monthlyDepAmt
from DEPOSIT
where status in ("APPROVED", "BOOKED")
and MONTH(date_created) = %s
and YEAR(date_created) = 2021
group by user_id) mD
on u.id = mD.user_id
left join (
select user_id, sum(amount) as monthlyWitAmt, max(is_full_withdrawal) as fullWit
from WITHDRAWAL
where status in ("APPROVED", "BOOKED")
and MONTH(date_created) = %s
and YEAR(date_created) = 2021
group by user_id) mW
on u.id = mW.user_id
left join (
select user_id, sum(amount) as cumDepAmt
from DEPOSIT
where status in ("APPROVED", "BOOKED")
and MONTH(date_created) <= %s
and YEAR(date_created) = 2021
group by user_id) cD
on u.id = cD.user_id
left join (
select user_id, sum(amount) as cumWitAmt
from WITHDRAWAL
where status in ("APPROVED", "BOOKED")
and MONTH(date_created) <= %s
and YEAR(date_created) = 2021
group by user_id) cW
on u.id = cW.user_id;
'''

df = pd.DataFrame(columns=column_names)

for i in range(len(months)):
    db_cursor.execute(query, (months[i], months[i], months[i] , months[i], months[i], months[i]))
    table_rows = db_cursor.fetchall()
    df2 = pd.DataFrame(table_rows, columns=column_names)
    df = pd.concat([df, df2])

df=df.sort_values(by=['UserID','Month'], ascending=True, ignore_index=True);

#Determine if user Churned
churnConditions = [
    (df['CumulativeWithdrawal'] == 0) | (df['CumulativeWithdrawal'] < df['CumulativeDeposit']),
    #(df['UserID'].shift() == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) #& ((df['Churn'].shift() == 'No') | (df['Churn'].shift() == 'Comeback')),
    (df['UserID'].shift() == df['UserID']) & (df['fullWithdrawal?'] == 1) #& ((df['Churn'].shift() == 'No') | (df['Churn'].shift() == 'Comeback')),
]
churnValues = ['No', 'Churn']
df['Churn'] = np.select(churnConditions, churnValues, default="No")

#Determine Churn Duration
churnTimeConditions = [
    (df['Churn'] == 'Churn') & (df['Churn'].shift() == 'No'),
    (df['UserID'].shift() == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift() == 'Churn') & (df['Churn'].shift(periods=2) == 'No'),
    (df['UserID'].shift(periods = 2) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 2) == 'Churn') & (df['Churn'].shift(periods=3) == 'No'),
    (df['UserID'].shift(periods = 3) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 3) == 'Churn') & (df['Churn'].shift(periods=4) == 'No'),
    (df['UserID'].shift(periods = 4) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 4) == 'Churn') & (df['Churn'].shift(periods=5) == 'No'),
    (df['UserID'].shift(periods = 5) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 5) == 'Churn') & (df['Churn'].shift(periods=6) == 'No'),
    (df['UserID'].shift(periods = 6) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 6) == 'Churn') & (df['Churn'].shift(periods=7) == 'No'),
    (df['UserID'].shift(periods = 7) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 7) == 'Churn') & (df['Churn'].shift(periods=8) == 'No'),
    (df['UserID'].shift(periods = 8) == df['UserID']) & (df['CumulativeWithdrawal'] > df['CumulativeDeposit']) & (df['Churn'].shift(periods = 8) == 'Churn'),
]
churnTimeValues = ['New Churn', '1 Month', '2 Month', '3 Month', '4 Month', '5 Month', '6 Month', '7 Month', '8 Month/more']
df['ChurnTimes'] = np.select(churnTimeConditions, churnTimeValues, default = "No")

#Determine if Comeback User
comebackConditions = [
    (df['UserID'].shift() == df['UserID']) & (df['CumulativeDeposit'] > df['CumulativeWithdrawal']) & (df['Churn'].shift() == 'Churn')
]
comebackValues = ['Comeback']
df['Comeback'] = np.select(comebackConditions, comebackValues, default = "No")

#Determine Duration before Comeback
comebackTimeConditions = [
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == 'New Churn'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '1 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '2 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '3 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '4 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '5 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '6 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '7 Month'),
    (df['Comeback'] == 'Comeback') & (df['ChurnTimes'].shift() == '8 Month/more')
]
comebackTimeValues = ['Next Month', '1 Month', '2 Month', '3 Month', '4 Month', '5 Month', '6 Month', '7 Month', '8 Month/more']
df['comebackTimes'] = np.select(comebackTimeConditions, comebackTimeValues, default = "No")

#df.query('UserID in [2474, 2475, 2515, 2616, 24495]').drop(columns=['Churn', 'Comeback'])

df.to_csv('churn_analysis_script2021_withEmail.csv', index=False)